In [18]:
import json
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.brkga.genetic import genetic as brkga
from src.brkga_variation.genetic import genetic as brkga_var
from src.s_genetic.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
source='worksat'
target='companyeconomicsector'

In [20]:
kb = json.loads(open('src/experiments/kb.txt').readline())

In [21]:
kb_source = kb['yago2s']
kb_target = kb['nell_finances']

In [22]:
yago_str = [['worksat(A, B)',
  {'': 'graduatedfrom(C, B), influences(C, D)',
   'true': 'hasacademicadvisor(E, C), hasacademicadvisor(C, F)',
   'true,false': 'iscitizenof(C, G), iscitizenof(A, G)',
   'true,false,false': 'haswebsite(D, H), diedin(C, I)',
   'false': 'hasacademicadvisor(A, J), hasgender(J, K)',
   'false,false': 'graduatedfrom(L, B), livesin(L, M)',
   'false,false,true': 'iscitizenof(A, M), iscitizenof(L, M)',
   'false,false,false': 'hasacademicadvisor(N, A), graduatedfrom(N, P)'},
  {'true,true': [0.388, 273, 1209],
   'true,false,true': [0.37, 69, 352],
   'true,false,false,true': [7.829, 88, 202],
   'true,false,false,false': [18.136, 687, 631],
   'false,true': [0.451, 865, 344],
   'false,false,true,true': [4.969, 33, 98],
   'false,false,true,false': [17.103, 703, 501],
   'false,false,false,true': [3.67, 85, 16],
   'false,false,false,false': [24.03, 1492, 942]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), influences(D, C)',
   'true': 'hasacademicadvisor(C, E), influences(F, D)',
   'true,false': 'iscitizenof(C, G), iscitizenof(A, G)',
   'true,false,false': 'iscitizenof(A, H), exports(H, I)',
   'false': 'hasacademicadvisor(J, A), hasgender(A, K)',
   'false,true': 'graduatedfrom(L, B), hasacademicadvisor(M, L)',
   'false,false': 'graduatedfrom(A, B)',
   'false,false,false': 'hasacademicadvisor(A, N), hasacademicadvisor(N, P)'},
  {'true,true': [0.378, 283, 1305],
   'true,false,true': [0.385, 79, 353],
   'true,false,false,true': [11.994, 386, 242],
   'true,false,false,false': [17.15, 545, 655],
   'false,true,true': [0.493, 44, 49],
   'false,true,false': [0.36, 417, 76],
   'false,false,true': [0.272, 0, 95],
   'false,false,false,true': [8.587, 290, 100],
   'false,false,false,false': [29.462, 2251, 1420]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), actedin(C, D)',
   'true': 'graduatedfrom(A, B)',
   'true,false': 'wrotemusicfor(E, D), directed(F, D)',
   'false': 'graduatedfrom(G, B), hasacademicadvisor(G, H)',
   'false,true': 'hasacademicadvisor(I, A), graduatedfrom(I, B)',
   'false,true,false': 'livesin(G, J), iscitizenof(A, J)',
   'false,false': 'hasacademicadvisor(K, A), hasgender(A, L)',
   'false,false,false': 'hasacademicadvisor(A, M), hasgender(M, N)'},
  {'true,true': [0.153, 4, 184],
   'true,false,true': [0.414, 433, 1308],
   'true,false,false': [0.49, 77, 67],
   'false,true,true': [0.064, 0, 141],
   'false,true,false,true': [6.93, 67, 220],
   'false,true,false,false': [20.31, 918, 826],
   'false,false,true': [0.362, 427, 79],
   'false,false,false,true': [11.452, 486, 182],
   'false,false,false,false': [27.442, 1883, 1288]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), hasacademicadvisor(D, C)',
   'true': 'graduatedfrom(A, B)',
   'true,false': 'owns(B, E), wasbornin(D, F)',
   'true,false,false': 'hasacademicadvisor(G, A), graduatedfrom(G, B)',
   'false': 'graduatedfrom(A, B)',
   'false,false': 'hasacademicadvisor(A, H), hasgender(H, I)',
   'false,false,true': 'hasacademicadvisor(J, A), graduatedfrom(A, K)',
   'false,false,false': 'graduatedfrom(L, B), haschild(M, L)'},
  {'true,true': [0.146, 5, 384],
   'true,false,true': [0.4, 322, 1131],
   'true,false,false,true': [0.635, 0, 98],
   'true,false,false,false': [18.546, 781, 735],
   'false,true': [0.129, 1, 106],
   'false,false,true,true': [5.613, 243, 38],
   'false,false,true,false': [13.303, 627, 258],
   'false,false,false,true': [10.404, 225, 233],
   'false,false,false,false': [28.122, 2091, 1312]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), actedin(C, D)',
   'true': 'edited(E, D), directed(E, F)',
   'true,false': 'owns(B, G), wasbornin(C, H)',
   'true,false,true': 'iscitizenof(A, I), islocatedin(D, I)',
   'false': 'graduatedfrom(A, B)',
   'false,false': 'graduatedfrom(A, J)',
   'false,false,true': 'owns(B, K), owns(J, L)',
   'false,false,false': 'graduatedfrom(M, B), isknownfor(M, N)'},
  {'true,true': [0.361, 154, 706],
   'true,false,true,true': [3.41, 16, 66],
   'true,false,true,false': [8.413, 145, 170],
   'true,false,false': [0.397, 199, 617],
   'false,true': [0.118, 2, 306],
   'false,false,true,true': [11.773, 283, 306],
   'false,false,true,false': [26.388, 2214, 1128],
   'false,false,false,true': [3.653, 19, 72],
   'false,false,false,false': [22.516, 1263, 924]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(A, B)',
   'true': 'hasgender(A, C), owns(B, D)',
   'false': 'graduatedfrom(E, B), actedin(E, F)',
   'false,true': 'haschild(G, E), wasbornin(E, H)',
   'false,true,true': 'livesin(I, H), hasgender(A, J)',
   'false,true,false': 'wrotemusicfor(K, F), ismarriedto(K, L)',
   'false,false': 'hasacademicadvisor(M, A), graduatedfrom(M, B)',
   'false,false,false': 'hasacademicadvisor(N, A), graduatedfrom(N, P)'},
  {'true,true': [0.163, 4, 183],
   'true,false': [0.111, 2, 307],
   'false,true,true,true': [5.377, 36, 271],
   'false,true,true,false': [1.339, 5, 4],
   'false,true,false,true': [12.385, 220, 730],
   'false,true,false,false': [11.418, 249, 370],
   'false,false,true': [0.096, 0, 95],
   'false,false,false,true': [8.864, 426, 113],
   'false,false,false,false': [35.62, 3353, 2222]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(C, B), influences(D, C)',
   'true': 'livesin(D, E), hasacademicadvisor(D, F)',
   'true,false': 'iscitizenof(C, G), iscitizenof(A, G)',
   'true,false,false': 'iscitizenof(A, H), exports(H, I)',
   'false': 'hasacademicadvisor(A, J), hasacademicadvisor(J, K)',
   'false,false': 'graduatedfrom(A, L), owns(L, M)',
   'false,false,true': 'hasacademicadvisor(N, A), iscitizenof(A, P)',
   'false,false,false': 'influences(A, Q), graduatedfrom(R, B)'},
  {'true,true': [0.345, 223, 1101],
   'true,false,true': [0.367, 90, 443],
   'true,false,false,true': [11.432, 399, 242],
   'true,false,false,false': [17.013, 581, 769],
   'false,true': [0.4, 500, 155],
   'false,false,true,true': [2.32, 41, 6],
   'false,false,true,false': [16.611, 802, 459],
   'false,false,false,true': [2.891, 47, 13],
   'false,false,false,false': [24.898, 1612, 1107]}],
 ['worksat(A, B)',
  {'': 'hasacademicadvisor(C, A), graduatedfrom(C, B)',
   'false': 'graduatedfrom(D, B), wrotemusicfor(D, E)',
   'false,true': 'directed(F, E), islocatedin(E, G)',
   'false,true,true': 'diedin(A, H), hascurrency(G, I)',
   'false,true,false': 'hasacademicadvisor(A, J), haswebsite(D, K)',
   'false,false': 'graduatedfrom(L, B), haschild(L, M)',
   'false,false,true': 'iscitizenof(L, N), iscitizenof(A, N)',
   'false,false,false': 'graduatedfrom(A, B)'},
  {'true': [0.085, 0, 241],
   'false,true,true,true': [2.65, 13, 35],
   'false,true,true,false': [6.432, 51, 415],
   'false,true,false,true': [2.016, 5, 50],
   'false,true,false,false': [6.758, 75, 167],
   'false,false,true,true': [4.937, 28, 239],
   'false,false,true,false': [19.173, 777, 1057],
   'false,false,false,true': [1.365, 1, 161],
   'false,false,false,false': [34.069, 3345, 1930]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(A, B)',
   'false': 'graduatedfrom(C, B), haschild(D, C)',
   'false,true': 'iscitizenof(C, E), iscitizenof(A, E)',
   'false,true,true': 'hascurrency(E, F), ismarriedto(G, D)',
   'false,true,false': 'iscitizenof(D, H), hasgender(A, I)',
   'false,false': 'graduatedfrom(A, J), owns(J, K)',
   'false,false,true': 'owns(B, L)',
   'false,false,false': 'graduatedfrom(M, B), owns(B, N)'},
  {'true': [0.13, 6, 490],
   'false,true,true,true': [3.882, 19, 112],
   'false,true,true,false': [5.293, 35, 297],
   'false,true,false,true': [9.176, 133, 478],
   'false,true,false,false': [17.117, 624, 748],
   'false,false,true,true': [10.951, 245, 269],
   'false,false,true,false': [15.82, 927, 360],
   'false,false,false,true': [11.022, 334, 241],
   'false,false,false,false': [26.87, 1972, 1300]}],
 ['worksat(A, B)',
  {'': 'graduatedfrom(A, B)',
   'false': 'hasacademicadvisor(A, C), graduatedfrom(D, B)',
   'false,true': 'iscitizenof(D, E), iscitizenof(A, E)',
   'false,true,true': 'dealswith(E, F)',
   'false,true,false': 'hasacademicadvisor(D, G), hasacademicadvisor(G, H)',
   'false,false': 'graduatedfrom(I, B), isknownfor(I, J)',
   'false,false,true': 'influences(K, I), influences(L, A)',
   'false,false,false': 'graduatedfrom(A, M)'},
  {'true': [0.126, 6, 490],
   'false,true,true,true': [3.795, 10, 149],
   'false,true,true,false': [6.43, 75, 199],
   'false,true,false,true': [10.456, 216, 395],
   'false,true,false,false': [8.926, 398, 107],
   'false,false,true,true': [1.073, 1, 39],
   'false,false,true,false': [7.528, 83, 313],
   'false,false,false,true': [27.948, 2231, 1502],
   'false,false,false,false': [23.126, 1275, 1101]}]]

In [23]:
pred_target = create_pred_target(kb_target)

In [24]:
import copy
src_struct = copy.deepcopy(yago_str)
new_src_struct = []
for i in range(0, len(src_struct)):
    new_src_struct.append(define_individual(src_struct[i], i))  
structured_src = src_struct

# target = f'recursion_{target}'

In [25]:
finances_dataset = datasets.load('nell_finances', kb_target, target=target, seed=441773, balanced=0)
train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(finances_dataset, 0)

0


In [26]:
facts = []
facts.extend(train_facts[0])
facts.extend(train_facts[1])
facts.extend(test_facts)
train_facts = [copy.deepcopy(facts), copy.deepcopy(facts)]
test_facts = [copy.deepcopy(facts)]

In [52]:
aa = open('train_facts_yago_nell.txt', 'w')
for i in train_facts:
    aa.write('[')
    for j in i:
        aa.write(j)
    aa.write(']')
aa.close()

In [53]:
aa = open('train_neg_yago_nell.txt', 'w')
for i in train_neg:
    aa.write('[')
    for j in i:
        aa.write(j)
    aa.write(']')
aa.close()

In [54]:
aa = open('train_pos_yago_nell.txt', 'w')
for i in train_pos:
    aa.write('[')
    for j in i:
        aa.write(j)
    aa.write(']')
aa.close()

In [56]:
aa = open('test_facts_yago_nell.txt', 'w')
for i in test_facts:
    aa.write('[')
    for j in i:
        aa.write(j)
    aa.write(']')
aa.close()

In [57]:
aa = open('test_neg_yago_nell.txt', 'w')
for i in test_neg:
    aa.write('[')
    for j in i:
        aa.write(j)
    aa.write(']')
aa.close()

In [58]:
aa = open('test_pos_yago_nell.txt', 'w')
for i in test_neg:
    aa.write('[')
    for j in i:
        aa.write(j)
    aa.write(']')
aa.close()

# Crossover original

## sem revisão

In [27]:
start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.0268040215553549
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
BEST: {'m_auc_pr': 0.770765, 'm_auc_roc': 0.9990399999999999, 'm_cll': -0.2371615, 'm_rec': 0.90625, 'm_pred': 0.356137, 'm_f1': 0.3413625, 's_auc_pr': 0.116622, 's_auc_roc': 0.0003369999999999762, 's_cll': 8.499999999994623e-06, 's_rec': 0.00919099999999995, 's_prec': 0.14802321186050518, 's_f1': 0.06550049999999999}
GENERATION:  1
MELHOR RESULTADO:  -1.0268040215553549
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
BEST: {'m_auc_pr': 0.770765, 'm_auc_roc': 0.9990399999999999, 'm_cll': -0.2371615, 'm_rec': 0.90625, 'm_pred': 0.356137, 'm_f1': 0.3413625, 's_auc_pr': 0.116622, 's_auc_roc': 0.0003369999999999762, 's_cll': 8.499999999994623e-06, 's_rec': 0.00919099999999995, 's_prec'

In [28]:
res

(<src.s_genetic.population.Population at 0x1281ab940>,
 [-1.0268040215553549,
  -1.0268040215553549,
  -1.0268040215553549,
  -1.0448098128411936,
  -1.0448098128411936,
  -1.0449003945023898,
  -1.0590050433129439,
  -1.0590050433129439,
  -1.0590050433129439,
  -1.0631084015312127,
  -1.0631084015312127,
  -1.0722734956443016,
  -1.0722734956443016,
  -1.0747421259717376,
  -1.0747421259717376],
 [{'m_auc_pr': 0.770765,
   'm_auc_roc': 0.9990399999999999,
   'm_cll': -0.2371615,
   'm_rec': 0.90625,
   'm_pred': 0.356137,
   'm_f1': 0.3413625,
   's_auc_pr': 0.116622,
   's_auc_roc': 0.0003369999999999762,
   's_cll': 8.499999999994623e-06,
   's_rec': 0.00919099999999995,
   's_prec': 0.14802321186050518,
   's_f1': 0.06550049999999999},
  {'m_auc_pr': 0.770765,
   'm_auc_roc': 0.9990399999999999,
   'm_cll': -0.2371615,
   'm_rec': 0.90625,
   'm_pred': 0.356137,
   'm_f1': 0.3413625,
   's_auc_pr': 0.116622,
   's_auc_roc': 0.0003369999999999762,
   's_cll': 8.499999999994623e-06,

In [29]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=False, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [30]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts[0]]

individuals = get_k_best_individuals(res[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.999962, 'AUC PR': 0.991709, 'CLL': -0.39305, 'Precision': [0.991453, 0.5], 'Recall': 0.991453, 'F1': 0.991453}, <boostsrl.boostsrl.test object at 0x12b9a0dc0>)
({'AUC ROC': 0.999988, 'AUC PR': 0.992614, 'CLL': -0.394699, 'Precision': [0.995536, 0.5], 'Recall': 0.952991, 'F1': 0.973799}, <boostsrl.boostsrl.test object at 0x12bb18eb0>)
INDIVIDUO  2
({'AUC ROC': 0.999962, 'AUC PR': 0.991709, 'CLL': -0.39305, 'Precision': [0.991453, 0.5], 'Recall': 0.991453, 'F1': 0.991453}, <boostsrl.boostsrl.test object at 0x12b9a0ca0>)
({'AUC ROC': 0.999988, 'AUC PR': 0.992614, 'CLL': -0.394699, 'Precision': [0.995536, 0.5], 'Recall': 0.952991, 'F1': 0.973799}, <boostsrl.boostsrl.test object at 0x12bb18eb0>)
INDIVIDUO  3
({'AUC ROC': 0.999962, 'AUC PR': 0.991709, 'CLL': -0.39305, 'Precision': [0.991453, 0.5], 'Recall': 0.991453, 'F1': 0.991453}, <boostsrl.boostsrl.test object at 0x12b9a0df0>)
({'AUC ROC': 0.999988, 'AUC PR': 0.992614, 'CLL': -0.394699, 'Precision': [0.995536,

## Aleatório

In [31]:
start = time.time()
res_a_c = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='random')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.0253245652037457
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.5426839999999999, 'm_auc_roc': 0.9977825, 'm_cll': -0.23605199999999998, 'm_rec': 1.0, 'm_pred': 0.269928, 'm_f1': 0.076653, 's_auc_pr': 0.00566899999999998, 's_auc_roc': 5.499999999991623e-06, 's_cll': 0.0023180000000000006, 's_rec': 0.0, 's_prec': 0.23007373333020872, 's_f1': 0.0023360000000000047}
GENERATION:  1
MELHOR RESULTADO:  -1.0323049592820184
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.0939415, 'm_auc_roc': 0.9853814999999999, 'm_cll': -0.30769599999999997, 'm_rec': 0.9191175, 'm_pred': 0.26380675, 'm_f1': 0.053616, 's_auc_pr': 0.023143499999999997, 's_auc_roc': 0.0008035000000000125, 's_cll': 0.0023669999999999802, 's_rec': 0.01102949999999

In [32]:
res_a_c

(<src.s_genetic.population.Population at 0x12b9a0f10>,
 [-1.0253245652037457,
  -1.0323049592820184,
  -1.0323049592820184,
  -1.0462927262388142,
  -1.0462927262388142,
  -1.0569583492739911,
  -1.0583166079436013,
  -1.059229887234707,
  -1.0592461751701066,
  -1.0662776432290044,
  -1.0671349737146656,
  -1.0699485217103906,
  -1.0699485217103906,
  -1.076775278511027,
  -1.0769638050316501],
 [{'m_auc_pr': 0.5426839999999999,
   'm_auc_roc': 0.9977825,
   'm_cll': -0.23605199999999998,
   'm_rec': 1.0,
   'm_pred': 0.269928,
   'm_f1': 0.076653,
   's_auc_pr': 0.00566899999999998,
   's_auc_roc': 5.499999999991623e-06,
   's_cll': 0.0023180000000000006,
   's_rec': 0.0,
   's_prec': 0.23007373333020872,
   's_f1': 0.0023360000000000047},
  {'m_auc_pr': 0.0939415,
   'm_auc_roc': 0.9853814999999999,
   'm_cll': -0.30769599999999997,
   'm_rec': 0.9191175,
   'm_pred': 0.26380675,
   'm_f1': 0.053616,
   's_auc_pr': 0.023143499999999997,
   's_auc_roc': 0.0008035000000000125,
   's_c

In [33]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=True, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [34]:
final_results = {}
final_results[f'{source}->{target}'] = []
final_results[f'{source}->{target}'].append(res)
best = final_results[f'{source}->{target}'][0]
for i in final_results[f'{source}->{target}']:
    if i[1][-1] > best[1][-1]:
        best = i

In [39]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts[0]]

individuals = get_k_best_individuals(res_a_c[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.979002, 'AUC PR': 0.281588, 'CLL': -0.406197, 'Precision': [0.027253, 0.5], 'Recall': 0.876068, 'F1': 0.052862}, <boostsrl.boostsrl.test object at 0x131bff760>)
({'AUC ROC': 0.975925, 'AUC PR': 0.13841, 'CLL': -0.407146, 'Precision': [0.025218, 0.5], 'Recall': 0.84188, 'F1': 0.048968}, <boostsrl.boostsrl.test object at 0x12b9a0e50>)
INDIVIDUO  2
({'AUC ROC': 0.975352, 'AUC PR': 0.071224, 'CLL': -0.431198, 'Precision': [0.025606, 0.5], 'Recall': 0.871795, 'F1': 0.04975}, <boostsrl.boostsrl.test object at 0x131bff9a0>)
({'AUC ROC': 0.975614, 'AUC PR': 0.067627, 'CLL': -0.431187, 'Precision': [0.025823, 0.5], 'Recall': 0.871795, 'F1': 0.05016}, <boostsrl.boostsrl.test object at 0x12b9a0e50>)
INDIVIDUO  3
({'AUC ROC': 0.975377, 'AUC PR': 0.07059, 'CLL': -0.431185, 'Precision': [0.025651, 0.5], 'Recall': 0.871795, 'F1': 0.049835}, <boostsrl.boostsrl.test object at 0x131bff910>)
({'AUC ROC': 0.975637, 'AUC PR': 0.067238, 'CLL': -0.431374, 'Precision': [0.025869, 0

## Guiado

In [36]:
start = time.time()
res_g_c = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='guided')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.0192928793512932
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.8897415, 'm_auc_roc': 0.997102, 'm_cll': -0.21153149999999998, 'm_rec': 1.0, 'm_pred': 0.267713, 'm_f1': 0.068428, 's_auc_pr': 0.019383499999999998, 's_auc_roc': 0.0004830000000000112, 's_cll': 0.0028014999999999984, 's_rec': 0.0, 's_prec': 0.23228718180196686, 's_f1': 0.0007660000000000028}
GENERATION:  1
MELHOR RESULTADO:  -1.0238604150675277
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.9953209999999999, 'm_auc_roc': 0.999994, 'm_cll': -0.21977750000000001, 'm_rec': 1.0, 'm_pred': 0.34630825, 'm_f1': 0.3228955, 's_auc_pr': 0.0008710000000000107, 's_auc_roc': 9.99

In [37]:
res_g_c

(<src.s_genetic.population.Population at 0x12f9fbf10>,
 [-1.0192928793512932,
  -1.0238604150675277,
  -1.024974331559698,
  -1.0264432656306437,
  -1.0264432656306437,
  -1.0382840153570698,
  -1.0382840153570698,
  -1.0382840153570698,
  -1.040135304493723,
  -1.040135304493723,
  -1.0401928985151023,
  -1.0401928985151023,
  -1.0401928985151023,
  -1.0401928985151023,
  -1.0401928985151023],
 [{'m_auc_pr': 0.8897415,
   'm_auc_roc': 0.997102,
   'm_cll': -0.21153149999999998,
   'm_rec': 1.0,
   'm_pred': 0.267713,
   'm_f1': 0.068428,
   's_auc_pr': 0.019383499999999998,
   's_auc_roc': 0.0004830000000000112,
   's_cll': 0.0028014999999999984,
   's_rec': 0.0,
   's_prec': 0.23228718180196686,
   's_f1': 0.0007660000000000028},
  {'m_auc_pr': 0.9953209999999999,
   'm_auc_roc': 0.999994,
   'm_cll': -0.21977750000000001,
   'm_rec': 1.0,
   'm_pred': 0.34630825,
   'm_f1': 0.3228955,
   's_auc_pr': 0.0008710000000000107,
   's_auc_roc': 9.999999999732445e-07,
   's_cll': 0.00335950

In [40]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts[0]]

individuals = get_k_best_individuals(res_g_c[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.999731, 'AUC PR': 0.867005, 'CLL': -0.292556, 'Precision': [0.034412, 0.5], 'Recall': 1.0, 'F1': 0.066534}, <boostsrl.boostsrl.test object at 0x131c79e50>)
({'AUC ROC': 0.999834, 'AUC PR': 0.989054, 'CLL': -0.28428, 'Precision': [0.037019, 0.5], 'Recall': 1.0, 'F1': 0.071396}, <boostsrl.boostsrl.test object at 0x12b9a0e50>)
INDIVIDUO  2
({'AUC ROC': 0.999731, 'AUC PR': 0.866292, 'CLL': -0.292559, 'Precision': [0.034417, 0.5], 'Recall': 1.0, 'F1': 0.066543}, <boostsrl.boostsrl.test object at 0x131c79820>)
({'AUC ROC': 0.999834, 'AUC PR': 0.989037, 'CLL': -0.28428, 'Precision': [0.037019, 0.5], 'Recall': 1.0, 'F1': 0.071396}, <boostsrl.boostsrl.test object at 0x12b9a0e50>)
INDIVIDUO  3
({'AUC ROC': 0.999731, 'AUC PR': 0.866292, 'CLL': -0.292559, 'Precision': [0.034417, 0.5], 'Recall': 1.0, 'F1': 0.066543}, <boostsrl.boostsrl.test object at 0x131c79dc0>)
({'AUC ROC': 0.999834, 'AUC PR': 0.989037, 'CLL': -0.28428, 'Precision': [0.037019, 0.5], 'Recall': 1.0, 'F1

## Mixed

In [41]:
start = time.time()
res_m_c = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='mixed')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.032958850158127
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.140689, 'm_auc_roc': 0.995154, 'm_cll': -0.276898, 'm_rec': 0.8878675, 'm_pred': 0.31910375, 'm_f1': 0.239172, 's_auc_pr': 0.013977000000000003, 's_auc_roc': 0.00022900000000003473, 's_cll': 0.00028100000000000347, 's_rec': 0.0018385000000000207, 's_prec': 0.18090475998488126, 's_f1': 0.0036489999999999995}
GENERATION:  1
MELHOR RESULTADO:  -1.032958850158127
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.140689, 'm_auc_roc': 0.995154, 'm_cll': -0.276898, 'm_rec': 0.8878675, 'm_pred': 0.31910375, 'm_f1': 0.239172, 's_auc_pr': 0.013977000000000003, 's_auc_roc': 0.00022900000000003473, 's_cll': 0.00028100000000000347, 's_rec': 0.00183850000

In [42]:
res_m_c

(<src.s_genetic.population.Population at 0x131cb9310>,
 [-1.032958850158127,
  -1.032958850158127,
  -1.0394823515146854,
  -1.0394823515146854,
  -1.0489383167833322,
  -1.048953043697381,
  -1.0490470397690708,
  -1.0490483822022938,
  -1.0491131663748197,
  -1.0491131663748197,
  -1.0491488027068656,
  -1.049152896695353,
  -1.049152896695353,
  -1.0491565641611837,
  -1.049165473315935],
 [{'m_auc_pr': 0.140689,
   'm_auc_roc': 0.995154,
   'm_cll': -0.276898,
   'm_rec': 0.8878675,
   'm_pred': 0.31910375,
   'm_f1': 0.239172,
   's_auc_pr': 0.013977000000000003,
   's_auc_roc': 0.00022900000000003473,
   's_cll': 0.00028100000000000347,
   's_rec': 0.0018385000000000207,
   's_prec': 0.18090475998488126,
   's_f1': 0.0036489999999999995},
  {'m_auc_pr': 0.140689,
   'm_auc_roc': 0.995154,
   'm_cll': -0.276898,
   'm_rec': 0.8878675,
   'm_pred': 0.31910375,
   'm_f1': 0.239172,
   's_auc_pr': 0.013977000000000003,
   's_auc_roc': 0.00022900000000003473,
   's_cll': 0.00028100000

In [43]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts[0]]

individuals = get_k_best_individuals(res_m_c[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'yago2s', 'nell_finances')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.996039, 'AUC PR': 0.156766, 'CLL': -0.328677, 'Precision': [0.161729, 0.5], 'Recall': 0.863248, 'F1': 0.272421}, <boostsrl.boostsrl.test object at 0x131f03340>)
({'AUC ROC': 0.996039, 'AUC PR': 0.156766, 'CLL': -0.327674, 'Precision': [0.161729, 0.5], 'Recall': 0.863248, 'F1': 0.272421}, <boostsrl.boostsrl.test object at 0x12b9a0e50>)
INDIVIDUO  2
({'AUC ROC': 0.996039, 'AUC PR': 0.156766, 'CLL': -0.328677, 'Precision': [0.161729, 0.5], 'Recall': 0.863248, 'F1': 0.272421}, <boostsrl.boostsrl.test object at 0x1321ceac0>)
({'AUC ROC': 0.996039, 'AUC PR': 0.156766, 'CLL': -0.327674, 'Precision': [0.161729, 0.5], 'Recall': 0.863248, 'F1': 0.272421}, <boostsrl.boostsrl.test object at 0x131c79340>)
INDIVIDUO  3
({'AUC ROC': 0.996038, 'AUC PR': 0.156682, 'CLL': -0.329025, 'Precision': [0.161729, 0.5], 'Recall': 0.863248, 'F1': 0.272421}, <boostsrl.boostsrl.test object at 0x1321ce5b0>)
({'AUC ROC': 0.996038, 'AUC PR': 0.156682, 'CLL': -0.327425, 'Precision': [0.1617

# Crossover modificado

## sem revisão

In [ ]:
start = time.time()
res_r_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_r_m

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_r_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

## Aleatório

In [ ]:
start = time.time()
res_a_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='random')
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_a_m

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_a_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

## Guiado

In [ ]:
start = time.time()
res_g_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='guided')
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_g_m

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_g_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

## Mixed

In [ ]:
start = time.time()
res_m_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='mixed')
print("FIM DO GROOT GENETIC: ", time.time()-start)

In [ ]:
res_m_m

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_m_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

In [60]:
aa = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)
aa

******************************************
Performing Parameter Learning
******************************************
Refine
['worksat(A, B)', {'': 'graduatedfrom(C, B), influences(C, D)', 'true': 'hasacademicadvisor(E, C), hasacademicadvisor(C, F)', 'true,false': 'iscitizenof(C, G), iscitizenof(A, G)', 'true,false,false': 'haswebsite(D, H), diedin(C, I)', 'false': 'hasacademicadvisor(A, J), hasgender(J, K)', 'false,false': 'graduatedfrom(L, B), livesin(L, M)', 'false,false,true': 'iscitizenof(A, M), iscitizenof(L, M)', 'false,false,false': 'hasacademicadvisor(N, A), graduatedfrom(N, P)'}, {'true,true': [0.388, 273, 1209], 'true,false,true': [0.37, 69, 352], 'true,false,false,true': [7.829, 88, 202], 'true,false,false,false': [18.136, 687, 631], 'false,true': [0.451, 865, 344], 'false,false,true,true': [4.969, 33, 98], 'false,false,true,false': [17.103, 703, 501], 'false,false,false,true': [3.67, 85, 16], 'false,false,false,false': [24.03, 1492, 942]}]
['worksat(A, B)', {'': 'graduatedfr

WILL Produced-Tree #1
% FOR companyeconomicsector(A, B):
%   if ( economicsectorcompany(B, C), acquired(D, C) )
%   then if ( bankboughtbank(C, E), bankboughtbank(F, C) )
%   | then return 0.6771965541471302;  // std dev = 3,945, 105,000 (wgt'ed) examples reached here.  /* #neg=19 #pos=86 */
%   | else if ( recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G) )
%   | | then return 0.8348931211460199;  // std dev = 2,965, 387,000 (wgt'ed) examples reached here.  /* #neg=9 #pos=378 */
%   | | else if ( bankchiefexecutiveceo(D, H) )
%   | | | then return -0.1418510649004878;  // std dev = 0,000, 13,000 (wgt'ed) examples reached here.  /* #neg=13 */
%   | | | else return -0.14185106490048793;  // std dev = 0,000, 135,000 (wgt'ed) examples reached here.  /* #neg=135 */
%   else if ( bankboughtbank(I, A), cityhascompanyoffice(J, I) )
%   | then return -0.14185106490048777;  // std dev = 0,000, 5,000 (wgt'ed) examples reached here.  /* #neg=5 */
%   | else if ( economi

Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.129455
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 34.87 seconds
Total inference time: 6.161 seconds
AUC ROC: 1.0


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.128551
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 9.767 seconds
Parameter learned model CLL: -0.128551


Structure after Parameter Learning
[['companyeconomicsector(A, B)', {'': 'economicsectorcompany(B, C), acquired(D, C)', 'true': 'bankboughtbank(C, E), bankboughtbank(F, C)', 'true,false': 'recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G)', 'true,false,false': 'bankchiefexecutiveceo(D, H)', 'false': 'bankboughtbank(I, A), cityhascompanyoffice(J, I)', 'false,false': 'economicsectorcompany(B, K)', 'false,false,true': 'recursion_companyeconomicsector(A, L), recursion_companyeconomicsector(K, L)'}, 

WILL Produced-Tree #1
% FOR companyeconomicsector(A, B):
%   if ( economicsectorcompany(B, C), acquired(D, C) )
%   then if ( bankboughtbank(C, E), bankboughtbank(F, C) )
%   | then if ( economicsectorcompany(B, A) )
%   | | then return 0.8581489350995121;  // std dev = 0,000, 86,000 (wgt'ed) examples reached here.  /* #pos=86 */
%   | | else return -0.14185106490048777;  // std dev = 0,000, 19,000 (wgt'ed) examples reached here.  /* #neg=19 */
%   | else if ( recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G) )
%   | | then if ( recursion_companyeconomicsector(H, B), companyceo(H, I) )
%   | | | then return 0.8469757507419664;  // std dev = 1,989, 358,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=354 */
%   | | | else return 0.6857351419960634;  // std dev = 2,034, 29,000 (wgt'ed) examples reached here.  /* #neg=5 #pos=24 */
%   | | else return -0.14185106490048804;  // std dev = 0,000, 148,000 (wgt'ed) examples reached here.  /* #neg=148 */
%   else if

Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.099913
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 182.22 seconds
Total inference time: 4.08 seconds
AUC ROC: 1.0


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.09952
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 6.593 seconds
Refined model CLL: -0.09952


Refining iteration 2
********************************
Candidate for revision
0;;companyeconomicsector(A, B) :- economicsectorcompany(B, C), acquired(D, C).;true;true
0;true;bankboughtbank(C, E), bankboughtbank(F, C).;true;true
0;true,true;economicsectorcompany(B, A).;true;true
0;true,false;recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G).;true;true
0;false;bankboughtbank(J, A), cityhascompanyoffice(K, J).;true;true
0;false,false;recursion_companyeconomicsector(A, B).;true;true
1;;companyeconomicsector(A, B

Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.09981
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 32.715 seconds
Total inference time: 3.042 seconds
AUC ROC: 1.0


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.099513
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 4.301 seconds
Refined model CLL: -0.099513


Refining iteration 3
********************************
Candidate for revision
0;;companyeconomicsector(A, B) :- economicsectorcompany(B, C), acquired(D, C).;true;true
0;true;bankboughtbank(C, E), bankboughtbank(F, C).;true;true
0;true,true;economicsectorcompany(B, A).;true;true
0;true,false;recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G).;true;true
0;false;bankboughtbank(I, A), cityhascompanyoffice(J, I).;true;true
0;false,false;recursion_companyeconomicsector(A, B).;true;true
1;;companyeconomicsector(A,

Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.09981
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 34.51 seconds
Total inference time: 6.057 seconds
AUC ROC: 1.0


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.099513
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 11.195 seconds
Refined model CLL: -0.099513


******************************************
Best model found
******************************************
Results
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.09981
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total learning time: 32.715 seconds
Total inference time: 3.042 seconds
AUC ROC: 1.0
Total revision time: 316.17099999999994
Best scored revision CLL: -0.099513




 {'AUC ROC': 1.0,
  'AUC PR': 1.0,
  'CLL': -0.09981,
  'Precision': [1.0, 0.5],
  'Recall': 1.0,
  'F1': 1.0,
  'Learning time': 32.715,
  'Inference time': 3.042},
 [['companyeconomicsector(A, B)',
   {'': 'economicsectorcompany(B, C), acquired(D, C)',
    'true': 'bankboughtbank(C, E), bankboughtbank(F, C)',
    'true,true': 'economicsectorcompany(B, A)',
    'true,false': 'recursion_companyeconomicsector(C, G), recursion_companyeconomicsector(A, G)',
    'true,false,true': 'companyceo(A, H)',
    'false': 'bankboughtbank(I, A), cityhascompanyoffice(J, I)',
    'false,false': 'recursion_companyeconomicsector(A, B)'},
   {'true,true,true': [0.0, 0, 86],
    'true,true,false': [0.0, 19, 0],
    'true,false,true,true': [1.684, 3, 52],
    'true,false,true,false': [2.427, 6, 326],
    'true,false,false': [0.0, 148, 0],
    'false,true': [0.0, 5, 0],
    'false,false,true': [0.0, 0, 80],
    'false,false,false': [2.48e-07, 363, 0]}],
  ['companyeconomicsector(A, B)',
   {'': 'economicsec

In [61]:
background_knowledge_src = boostsrl.modes(kb_target, [target], useStdLogicVariables=False, 
                                      maxTreeDepth=3, nodeSize=2, numOfClauses=8)
sys.path.insert(0, '../TreeBoostler')

from revision import revision

def print_function(message):
        global experiment_title
        global nbr
        experiment_title = f'Twitter->Yeast2: {source}->{target}'
        nbr = 1
        if not os.path.exists('experiments/' + experiment_title):
            os.makedirs('experiments/' + experiment_title)
        with open('experiments/' + experiment_title + '/' + str(nbr) + '_' + experiment_title + '.txt', 'a') as f:
            print(message, file=f)
            print(message)

model = revision.learn_test_model(background_knowledge_src, boostsrl, target, tr_train_pos, tr_train_neg, copy.deepcopy(facts), test_pos, test_neg, copy.deepcopy(facts), trees=10, print_function=print_function)

WILL Produced-Tree #1
% FOR companyeconomicsector(A, B):
%   if ( recursion_companyeconomicsector(A, B) )
%   then return 0.858148935099508;  // std dev = 2,57e-06, 544,000 (wgt'ed) examples reached here.  /* #pos=544 */
%   else return -0.14185106490048816;  // std dev = 0,000, 544,000 (wgt'ed) examples reached here.  /* #neg=544 */
WILL Produced-Tree #2
% FOR companyeconomicsector(A, B):
%   if ( recursion_companyeconomicsector(A, B) )
%   then return 0.7194734122109585;  // std dev = 0,000, 544,000 (wgt'ed) examples reached here.  /* #pos=544 */
%   else return -0.12544463852839233;  // std dev = 0,000, 544,000 (wgt'ed) examples reached here.  /* #neg=544 */
WILL Produced-Tree #3
% FOR companyeconomicsector(A, B):
%   if ( recursion_companyeconomicsector(A, B) )
%   then return 0.555366436746283;  // std dev = 8,09e-07, 544,000 (wgt'ed) examples reached here.  /* #pos=544 */
%   else return -0.1123163781936057;  // std dev = 0,000, 544,000 (wgt'ed) examples reached here.  /* #neg=54